In [1]:
import pandas as pd

# Suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [2]:
import requests # For making HTTP requests to the API
import datadotworld as dw


api_token = '*****'
dataset_key = 'markbradbourne/rwfd-real-world-fake-data'

# Set up headers for API requests
headers = {
    'Authorization': f'Bearer {api_token}' # Bearer token authentication. The Bearer token is a secure way to authenticate API requests.
}

try:
    # Get dataset metadata
    response = requests.get(
        f'https://api.data.world/v0/datasets/{dataset_key}',
        headers=headers
    )
    
    # Get the data file
    download_url = f'https://download.data.world/file_download/{dataset_key}/Hospital%20ER.csv'
    response = requests.get(download_url, headers=headers)
    
    # Load into pandas
    # Uses StringIO to create a text stream from the response. StringIO is used because pd.read_csv() expects either a file path or a file-like object
    df = pd.read_csv(pd.io.common.StringIO(response.text))
    
    #print("\nDataframe Preview:")
    #print(df.head())
    print("Dataset downloaded Successfully")
    
    # Save locally
    df.to_csv('downloaded_dataframe.csv', index=False)

except Exception as e:
    print(f"Error: {e}")
    import traceback
    print("\nFull error traceback:")
    print(traceback.format_exc())

Dataset downloaded Successfully


In [2]:
pr = pd.read_csv('downloaded_dataframe.csv')
pr.head(10)

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,patient_race,patient_admin_flag,patient_waittime,department_referral
0,2020-03-20 08:47:01,145-39-5406,M,69,10.0,H,Glasspool,White,False,39,None
1,2020-06-15 11:29:36,316-34-3057,M,4,NaN,X,Methuen,Native American/Alaska Native,True,27,None
2,2020-06-20 09:13:13,897-46-3852,F,56,9.0,P,Schubuser,African American,True,55,General Practice
3,2020-02-04 22:34:29,358-31-9711,F,24,8.0,U,Titcombe,Native American/Alaska Native,True,31,General Practice
4,2020-09-04 17:48:27,289-26-0537,M,5,NaN,Y,Gionettitti,African American,False,10,Orthopedics
5,2019-04-20 00:13:05,255-51-2877,M,58,NaN,H,Buff,Asian,False,59,None
6,2019-08-23 08:26:21,465-97-0990,F,68,NaN,F,Perrat,White,True,43,None
7,2019-07-29 16:57:15,157-31-7520,F,47,NaN,K,Gwillim,Two or More Races,True,23,None
8,2020-02-19 06:54:39,432-34-5614,F,79,1.0,E,Dewhirst,White,False,42,None
9,2020-10-11 05:25:17,609-17-8678,M,62,NaN,M,Crebo,African American,False,51,None


In [3]:
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  9216 non-null   object 
 1   patient_id            9216 non-null   object 
 2   patient_gender        9216 non-null   object 
 3   patient_age           9216 non-null   int64  
 4   patient_sat_score     2517 non-null   float64
 5   patient_first_inital  9216 non-null   object 
 6   patient_last_name     9216 non-null   object 
 7   patient_race          9216 non-null   object 
 8   patient_admin_flag    9216 non-null   bool   
 9   patient_waittime      9216 non-null   int64  
 10  department_referral   9216 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 729.1+ KB


In [4]:
# checking patient score values
pr.patient_sat_score.unique()

array([10., nan,  9.,  8.,  1.,  2.,  4.,  0.,  7.,  3.,  6.,  5.])

In [5]:
total_nan_counts = pr.patient_sat_score.isna().sum()
percentage = round((total_nan_counts / len(pr)) * 100)
print (f"Number of No rating given: {total_nan_counts}")
print(f"Percentange of no ratings given: {percentage}%")



Number of No rating given: 6699
Percentange of no ratings given: 73%


In [6]:
#Combining first name and Last name columns into one

pr['patient_fullName'] = pr['patient_first_inital'] + '. ' + pr['patient_last_name']
# pr['fullName'] = pr.iloc[:,5] + '. ' + pr.iloc[:,6]
pr.sample()



,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,patient_race,patient_admin_flag,patient_waittime,department_referral,patient_fullName
7757,2020-05-20 03:10:51,642-24-2097,M,54,NaN,F,Dienes,White,False,40,None,F. Dienes


In [7]:
# Drop patient first initial and patient last name columns

pr.drop(columns=['patient_first_inital', 'patient_last_name'], inplace=True)
pr.sample()

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_race,patient_admin_flag,patient_waittime,department_referral,patient_fullName
2005,2019-08-21 09:47:52,168-19-7400,F,49,8.0,White,True,43,Orthopedics,W. Grebbin


In [8]:
#convert order date from object data type to datetime
pr['date'] = pd.to_datetime(pr['date'], format='%Y-%m-%d')
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 9216 non-null   datetime64[ns]
 1   patient_id           9216 non-null   object        
 2   patient_gender       9216 non-null   object        
 3   patient_age          9216 non-null   int64         
 4   patient_sat_score    2517 non-null   float64       
 5   patient_race         9216 non-null   object        
 6   patient_admin_flag   9216 non-null   bool          
 7   patient_waittime     9216 non-null   int64         
 8   department_referral  9216 non-null   object        
 9   patient_fullName     9216 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 657.1+ KB


In [ ]:
#load the data into sql server using sqlalchemy
import sqlalchemy as sal
engine = sal.create_engine(
    "mssql+pyodbc://GLADYS-PC/BI_Project?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes"
)
conn=engine.connect()

In [11]:
#load the data into sql server using append option
pr.to_sql('hospital_er', con=conn , index=False, if_exists = 'append')